In [85]:
from pinecone import Pinecone, ServerlessSpec
import os
import time
from openai import OpenAI
import pandas as pd
import dotenv as dotenv

dotenv.load_dotenv()

True

In [86]:
# model_name = os.getenv("MODEL_NAME")
pinecone_api_key = os.getenv("PINECONE_API_KEY")
# index_name = os.getenv("PINECONE_INDEX_NAME")
index_name = "cofeeshop"

In [87]:
index_name

'cofeeshop'

In [88]:
# Using local Python embedding (using 'all-MiniLM-L6-v2' model from 'SentenceTransformer' library)

# can do on Ollama (using model like nomic-embed-text) or cloud like RunPod as well

from sentence_transformers import SentenceTransformer

model = SentenceTransformer('all-MiniLM-L6-v2')

In [89]:
pc = Pinecone(
    api_key=pinecone_api_key,
)

client = OpenAI(
    base_url="http://localhost:11434/v1",
    api_key="ollama"
)

# Try out embedding

In [90]:
text = "hello there"
embeddings = model.encode(text).tolist()
print(embeddings)

[-0.09443895518779755, 0.07322290539741516, 0.04226871579885483, 0.054307498037815094, -0.0671972930431366, -0.08545089513063431, 0.05023467540740967, 0.05221310257911682, -0.033030565828084946, -0.01926741562783718, 0.019890714436769485, -0.018779389560222626, -0.04508374631404877, -0.023084456101059914, -0.008017867803573608, -0.0005378724890761077, 0.01777522824704647, -0.007733900565654039, -0.1338098645210266, 0.0024418311659246683, -0.03394952043890953, 0.06793279200792313, -0.08245820552110672, 0.051667749881744385, -0.04819176346063614, -0.0625123456120491, 0.031222902238368988, 0.02164979837834835, -0.009581792168319225, -0.04121411591768265, 0.014142862521111965, 0.04402715712785721, 0.04060816392302513, 0.044027648866176605, 0.04674626514315605, 0.056684963405132294, -0.097380630671978, -0.059672437608242035, 0.05624126270413399, -0.010554773733019829, 0.018759246915578842, -0.049062665551900864, 0.001644360600039363, -0.014013580046594143, 0.04789145663380623, -0.0865960046

In [91]:
len(embeddings)

384

# Wrangle dataset

In [92]:
df = pd.read_json("products/products.jsonl", lines=True)

In [93]:
df.head(5)

,name,category,description,ingredients,price,rating,image_path
0,Cappuccino,Coffee,A rich and creamy cappuccino made with freshly...,"[Espresso, Steamed Milk, Milk Foam]",4.50,4.7,cappuccino.jpg
1,Jumbo Savory Scone,Bakery,"Deliciously flaky and buttery, this jumbo savo...","[Flour, Butter, Cheese, Herbs, Baking Powder, ...",3.25,4.3,SavoryScone.webp
2,Latte,Coffee,"Smooth and creamy, our latte combines rich esp...","[Espresso, Steamed Milk, Milk Foam]",4.75,4.8,Latte.jpg
3,Chocolate Chip Biscotti,Bakery,"Crunchy and delightful, this chocolate chip bi...","[Flour, Sugar, Chocolate Chips, Eggs, Almonds,...",2.50,4.6,chocolat_biscotti.jpg
4,Espresso shot,Coffee,"A bold shot of rich espresso, our espresso is ...",[Espresso],2.00,4.9,Espresso_shot.webp


In [94]:
df['text'] = df['name'] + " : " + df['description'] + " -- ingredients: " + df['ingredients'].astype(str) + " -- price: " + df['price'].astype(str) + " -- rating: " + df['rating'].astype(str)

In [95]:
df['text'].head(2)

0    Cappuccino : A rich and creamy cappuccino made...
1    Jumbo Savory Scone : Deliciously flaky and but...
Name: text, dtype: object

In [96]:
texts = df['text'].tolist()

In [97]:
texts[:2]

["Cappuccino : A rich and creamy cappuccino made with freshly brewed espresso, steamed milk, and a frothy milk cap. This delightful drink offers a perfect balance of bold coffee flavor and smooth milk, making it an ideal companion for relaxing mornings or lively conversations. -- ingredients: ['Espresso', 'Steamed Milk', 'Milk Foam'] -- price: 4.5 -- rating: 4.7",
 "Jumbo Savory Scone : Deliciously flaky and buttery, this jumbo savory scone is filled with herbs and cheese, creating a mouthwatering experience. Perfect for a hearty snack or a light lunch, it pairs beautifully with your favorite coffee or tea. -- ingredients: ['Flour', 'Butter', 'Cheese', 'Herbs', 'Baking Powder', 'Salt'] -- price: 3.25 -- rating: 4.3"]

In [98]:
with open("products/Merry's_way_about_us.txt") as f:
    Merry_way_about_section = f.read()
    
Merry_way_about_section = "Coffee shop Merry's Way about section: " + Merry_way_about_section
texts.append(Merry_way_about_section)

In [99]:
with open('products/menu_items_text.txt') as f:
    menu_items_text = f.read()
    
menu_items_text = "Menu Items: " + menu_items_text
texts.append(menu_items_text)

In [100]:
texts

["Cappuccino : A rich and creamy cappuccino made with freshly brewed espresso, steamed milk, and a frothy milk cap. This delightful drink offers a perfect balance of bold coffee flavor and smooth milk, making it an ideal companion for relaxing mornings or lively conversations. -- ingredients: ['Espresso', 'Steamed Milk', 'Milk Foam'] -- price: 4.5 -- rating: 4.7",
 "Jumbo Savory Scone : Deliciously flaky and buttery, this jumbo savory scone is filled with herbs and cheese, creating a mouthwatering experience. Perfect for a hearty snack or a light lunch, it pairs beautifully with your favorite coffee or tea. -- ingredients: ['Flour', 'Butter', 'Cheese', 'Herbs', 'Baking Powder', 'Salt'] -- price: 3.25 -- rating: 4.3",
 "Latte : Smooth and creamy, our latte combines rich espresso with velvety steamed milk, creating a perfect balance of flavor and texture. Enjoy it as a comforting treat any time of day, whether you're starting your morning or taking a midday break. -- ingredients: ['Espre

# Generating Embeddings

In [101]:
embeddings = model.encode(texts).tolist()

In [102]:
embeddings[0]

[0.03352544456720352,
 -0.057899560779333115,
 -0.0017868939321488142,
 0.05488213151693344,
 0.05808444693684578,
 -0.000260431959759444,
 0.01678721234202385,
 0.042207978665828705,
 -0.022465772926807404,
 -0.0008331801509484649,
 -0.00828015711158514,
 -0.0929771363735199,
 -0.014481251128017902,
 -0.01984688639640808,
 -0.015685463324189186,
 -0.07400842010974884,
 0.026576487347483635,
 -0.002823738381266594,
 0.0970730185508728,
 0.0202372707426548,
 0.006772961933165789,
 -0.055288415402173996,
 0.03207574412226677,
 0.10568636655807495,
 0.10774405300617218,
 0.09208441525697708,
 0.0501842126250267,
 0.02534065954387188,
 -0.04620799049735069,
 -0.07177940756082535,
 -0.05219711363315582,
 -0.06340939551591873,
 0.06747890263795853,
 -0.028087187558412552,
 -0.06899979710578918,
 0.08348885923624039,
 0.10179386287927628,
 -0.04042473062872887,
 0.057865243405103683,
 0.0008215632988139987,
 -0.010293971747159958,
 0.037107039242982864,
 0.09358837455511093,
 0.02118055708706

# Push data to Vector DB

In [104]:
pc.create_index(
    name=index_name,
    dimension=384,
    metric="cosine",
    spec=ServerlessSpec(
        cloud="aws",
        region="us-east-1",
    )
)

In [105]:
# Wait for index to ready

while not pc.describe_index(index_name).status['ready']:
    time.sleep(1)

index = pc.Index(index_name)

vectors = []

for text, e in zip(texts, embeddings):
    entry_id = text.split(":")[0]
    vectors.append({
        "id": entry_id, 
        "values": e, 
        "metadata": {"text": text}
    })

index.upsert(vectors=vectors, namespace='ns1') 

{'upserted_count': 20}

# Get closest document

In [106]:
# embeddings = model.encode("How much is Cappuccino?").tolist()

embeddings = model.encode("Chocolate Biscuit available?").tolist()

In [ ]:
results = index.query(
    namespace='ns1',
    vector=embeddings,
    top_k=2,     # closest 2 results
    include_values=False,
    include_metadata=True
)

In [112]:
results

{'matches': [{'id': 'Chocolate Chip Biscotti ',
              'metadata': {'text': 'Chocolate Chip Biscotti : Crunchy and '
                                   'delightful, this chocolate chip biscotti '
                                   'is perfect for dipping in your coffee or '
                                   'enjoying on its own. Each bite offers a '
                                   'satisfying crunch and a burst of rich '
                                   'chocolate, making it a favorite for any '
                                   "biscotti lover. -- ingredients: ['Flour', "
                                   "'Sugar', 'Chocolate Chips', 'Eggs', "
                                   "'Almonds', 'Baking Powder'] -- price: 2.5 "
                                   '-- rating: 4.6'},
              'score': 0.641696036,
              'values': []},
             {'id': 'Menu Items',
              'metadata': {'text': 'Menu Items: Menu Items\n'
                                   '\